In [226]:
%run Comment_Questions.ipynb # Exectute temporary training Parameters

# NEU Unit (Reconfiguration Map)

## Initialize Modules

###### Imports

In [227]:
# Deep Learning & ML
import tensorflow as tf
import keras as Kr
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils

# General
import numpy as np
import time

# Message Boxes
import tkinter
from tkinter import messagebox

### Define Reconfiguration Map

1. First applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.

2. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}$
3. Adds trainable $d\times d$ (weight) matrix $A$ and trainable bias (vector) $b \in \mathbb{R}^d$ to that output:

In [228]:
class reconfiguration(tf.keras.layers.Layer):

    def __init__(self, *args, **kwargs):
        super(BiasLayer, self).__init__(*args, **kwargs)
    
    def build(self, input_shape):
        # Initialize trainable Linear Endomorphism in Tangent Space
        self.weight = self.add_weight(shape=input_shape[1],trainable=True)
        # Initialize trainable Affine Shift in Tangent Space
        self.bias = self.add_weight(shape=input_shape[1:],trainable=True)
        # location parameter
        self.location = self.add_weight(shape=input_shape[1:],trainable=True)
        # scaling parameter
        self.scale = self.add_weight(shape=input_shape[1:],trainable=True)
        #
        super(reconfiguration, self).build(input_shape)

    def call(self, x):
        # Shift and scale data
        #---------------------#
        # Shift 
        x_shift = x - self.location
        # Rescale
        x = tf.math.multiply(x_shift,self.scale)
        
        # Apply bumpy function & tangent space transformation
        #----------------------------------------------------#
        # 1) Bump Function
        x_in_abs = tf.math.abs(x)
        x_thresheld = tf.where(tf.math.abs(x) >= 1, 0 , tf.math.exp(tf.pow(1-x,-1)))
        # 2) Map to Diagonal Matrix & Apply Bias
        x_out = tf.expand_dims((x_thresheld + self.bias), 1) 
        # 3) Apply weights
        x_out = (x_out * self.weight) 
        # 4) Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # Mulpliply Transformation with shifted data
        #--------------------------------------------#
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # Recenter Data
        #---------------#
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out
        
#    def reconfiguration(self):
#        config = super(bump_function_layer, self).get_config()
#        config['dim'] = self.dim
#        return config

In [229]:
input_layer = tf.keras.Input(shape=[5])
x  = reconfiguration()(input_layer)
model = tf.keras.Model(inputs=[input_layer], outputs=[x])

model.summary()

TypeError: super(type, obj): obj must be an instance or subtype of type

### Build Reconfiguration

In [298]:
import tensorflow as tf
print('TensorFlow:', tf.__version__)

class BiasLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(BiasLayer, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.weight = self.add_weight(shape=[input_shape[1],input_shape[1]], # CURRENTLY NOT A MATRIX FIX THIS!!!
                                    initializer='zeros',
                                    trainable=True)
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.location = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.scale = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        
        
        
    def call(self, x):
        # Shift and scale data
        #---------------------#
        # Shift 
        x_shift = x - self.location
        # Rescale
        x_mod = tf.math.multiply(x_shift,self.scale)
        
        # Apply bumpy function & tangent space transformation
        #----------------------------------------------------#
        # 1) Bump Function
        x_in_abs = tf.math.abs(x_mod)
        x_thresheld = tf.where(x_in_abs >= 1, 0 ,tf.math.exp(tf.pow(1-x_in_abs,-1)) ) #2 =tf.math.exp(tf.pow(1-x_in_abs,-1)) 
        # 2) Map to Diagonal Matrix & Apply Bias
        x_out = tf.expand_dims((x_in_abs + self.bias), 1) 
        # 3) Apply weights
        x_out = (x_out * self.weight) 
        # 4) Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # Mulpliply Transformation with shifted data
        #--------------------------------------------#
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # Recenter Data
        #---------------#
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out

input_layer = tf.keras.Input(shape=[5])
x  = BiasLayer()(input_layer)
model = tf.keras.Model(inputs=[input_layer], outputs=[x])

model.summary()

TensorFlow: 2.1.0


TypeError: in converted code:

    <ipython-input-294-37481791d4b2>:36 call  *
        x_thresheld = tf.where(x_in_abs >= 1, 0 ,tf.math.exp(tf.pow(1-x_in_abs,-1)) )
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/array_ops.py:3930 where_v2
        return gen_math_ops.select_v2(condition=condition, t=x, e=y, name=name)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_math_ops.py:8692 select_v2
        "SelectV2", condition=condition, t=t, e=e, name=name)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py:504 _apply_op_helper
        inferred_from[input_arg.type_attr]))

    TypeError: Input 'e' of 'SelectV2' Op has type float32 that does not match type int32 of argument 't'.


In [304]:
x_in_abs=tf.constant([1,2,.1])
x_thresheld = tf.where(x_in_abs >= 1, 0 ,tf.math.exp(tf.pow(1-x_in_abs,-1)) )
def test_f(x): tf.math.exp(1-x)
tf.math.exp(tf.pow(1-x_in_abs,-1))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([       inf, 0.36787945, 3.037732  ], dtype=float32)>